<a href="https://colab.research.google.com/github/uoahy-6uoas/HG-Machine/blob/main/hg5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

keywords = 앙상블 학습, 랜덤포레스트, 엑스트라트리, 그레디언트부스팅

**앙상블학습**
>  앙상블 학습(Ensemble Learning)은 여러 개의 분류기를 생성하고, 그 예측을 결합함으로써 보다 정확한 예측을 도출하는 기법을 말합니다. 강력한 하나의 모델을 사용하는대신 보다 약한 모델 여러개를 조합하여 더 정확한 예측에 도움을 주는 방식입니다. *DINNOPARTNERS*

**랜덤포레스트**
> 앙상블학습의 대표주자, 하나의 결과에 도달하기 위해 여러 의사결정 트리의 출력을 결합함 *IBM*

부트스트랩 샘플 = 훈련샘플에서 중복을 허락하여 n개 뽑아 훈련시키기를 여러번 반복

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
#교차검증
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)

In [3]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True,n_jobs=-1)
np.mean(scores['train_score']) , np.mean(scores['test_score'])

(0.9973541849413872, 0.8914794180795143)

In [4]:
#특성중요도
rf.fit(train_input,train_target)
rf.feature_importances_
#sugar importances decreased
#하나의 특성에 집중되지 않아 과대적합을 줄일 수 있는 것이 장점.

array([0.23645331, 0.49588573, 0.26766095])

In [5]:
#rf는 자체적으로 모델을 평가할 수 있음, oob는 부트스트랩 샘플에 포함되지 않는 샘플들 = 훈련 안시킨 샘플
rf = RandomForestClassifier(oob_score=True,n_jobs=-1)
rf.fit(train_input, train_target)
rf.oob_score_ #각 결정트리의 oob점수를 평균하여 출력

0.8995574369828747

**엑스트라트리**
>  엑스트라 트리는 포레스트 트리의 각 후보 특성을 무작위로 분할하는 식으로 무작위성을 증가 시킵니다. *텐서플로우블로그*

splitter = 'random'인 경우

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1)
scores = cross_validate(et, train_input, train_target, return_train_score=True,n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])
# 일단 무작위로 분할하기 때문에 속도가 빠름.

(0.9974503966084433, 0.8882081513289405)

In [7]:
et.fit(train_input,train_target)
et.feature_importances_
#역시 당도가 낮아진 모습

array([0.18138546, 0.53225678, 0.28635776])

**그레디언트 부스팅**
> depth가 얕은 결정트리를 사용하여 이진트리의 오차를 보완하는 방식으로 앙상블 *SAS*<br>
회귀에는 평균제곱오차함수, 분류에는 로지스틱손실함수 사용

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])
#깊이가 얖아서 과대적합이 일어나지 않는 모습

(0.8881086892152563, 0.8720430147331015)

In [9]:
#학습률(경사하강법의 보폭)을 늘려보자
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9464595437171814, 0.8780082549788999)

In [10]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15833382, 0.68009223, 0.16157395])

In [11]:
#히스토그램 기반 그레디언트 부스팅 : 설명을 들어도 와닿지 않기 시작함.. 수식으로 나중에 공부하자
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier()
scores = cross_validate(hgb,train_input,train_target,return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9321723946453317, 0.8801241948619236)

In [12]:
#feature_importances_
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
result = permutation_importance(hgb,train_input,train_target,n_repeats=10,n_jobs=-1)
result.importances_mean #[특성중요도, 평균, 표준편차]

array([0.08984029, 0.23756013, 0.07994997])

In [13]:
hgb.score(test_input,test_target)

0.8723076923076923

In [14]:
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier()
scores = cross_validate(xgb,train_input,train_target,return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9558401176154364, 0.8774313319019769)

In [15]:
#LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
scores = cross_validate(lgb,train_input,train_target,return_train_score=True,n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.935828414851749, 0.8801251203079884)

* 확인문제
1. 여러 개의 모델을 훈련시키고 각 모델의 예측을 취합하여 최종 결과를 만드는 학습 방식은 앙상블학습
2. 비정형 데이터에 속하는 것은 이미지 데이터
3. 기본적으로 부트스트랩 샘플을 사용하는 알고리즘은 랜덤 포레스트, 엑스트라트리는 기본적으로 bootstrap=False임

ch5 fin....